<a href="https://colab.research.google.com/github/sdam-au/PHI_ETL/blob/master/scripts/1_3_EXTRACTING_LOCATIONS_AND_DATES_FROM_RAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import math
import pandas as pd

import sys
### we do a lot of requests during the scrapping. Some of them with requests package, some of them with urllib
import requests
from urllib.request import urlopen 
from urllib.parse import quote  
from bs4 import BeautifulSoup
import xml.etree.cElementTree as ET

import zipfile
import io

# to avoid errors, we sometime use time.sleep(N) before retrying a request
import time
# the input data have typically a json structure
import json
import getpass

import datetime as dt
# for simple paralel computing:
from concurrent.futures import ThreadPoolExecutor

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [2]:
!pip install sddk
import sddk

In [3]:
conf = sddk.configure("SDAM_root", "648597@au.dk")

sciencedata.dk username (format '123456@au.dk'): 648597@au.dk
sciencedata.dk password: ··········
connection with shared folder established with you as its owner
endpoint variable has been configured to: https://sciencedata.dk/files/SDAM_root/


In [0]:
### authorize google sheets
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
# establish connection with particular sheet by its url:
PHI_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1zfTw0Hf304maBmrYvaMxRLnv1zfAVFixrtGTTsLCcT4/edit?usp=sharing")

# Read data

In [0]:
PHI = sddk.read_file("SDAM_data/PHI/PHI_raw.json", "df", conf)

In [8]:
PHI.head(5)

,URL,Book,Text,hdr1,hdr2,tildeinfo,note,lines,metadata,data,filename,PHI_ID
0,/text/237443?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 169,3",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 169,3",Att. — Athens: Akropolis — stoich. 28 — 303/2 BC,None,4,1\n\n\n,ἐπὶ Λε[ωστράτου ἄρχοντος ἐπὶ τῆς Κ]-\nεκρο[πίδ...,CSCA.csv,237443
1,/text/237444?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 169,4",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 169,4",Att. — Athens: EM — stoich. 35 — 306-302 BC,None,15,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15,[․]ομε̣[․․․․․․․․․․․․․28․․․․․․․․․․․․․ τῆς]\nπρυ...,CSCA.csv,237444
2,/text/237445?location=1365&patt=&bookid=411&bo...,CSCA,"5 (1972) 173,5",Regions\nAttica (IG I-III)\nAttica,"CSCA\n5 (1972) 173,5",Att. — Athens — non-stoich.,None,16,1\n\n\n\n5\n\n\n\n\n10\n\n\n\n\n15\n,— — — — —υτω․․․․11․․․․․\n— — — — —ωντωι̣․․․․9․...,CSCA.csv,237445
3,/text/237446?location=1365&patt=&bookid=411&bo...,CSCA,"9 (1976) 9,4",Regions\nAttica (IG I-III)\nAttica,"CSCA\n9 (1976) 9,4",Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26.1...,None,2,A.1\nB.1,ψῆφος ⋮ δημοσία.\nΔ ․,CSCA.csv,237446
4,/text/237447?location=1365&patt=&bookid=411&bo...,CSCA,"9 (1976) 11,16",Regions\nAttica (IG I-III)\nAttica,"CSCA\n9 (1976) 11,16",Att. — Athens: Eth.Mus. — 4th c. BC — SEG 26.1...,None,2,A.1\nB.1,ψῆφος ⋮ δημοσία.\nΓ Ε,CSCA.csv,237447


In [0]:
def date_extractor(datation):
    ### simple cleaning?
    datation = re.sub("\/\d+", "", datation)
    for element in ["?"]:
        datation = datation.replace(element, "")
    for element in ["AD", "Rom. Imp.", "Byzant", "Roman period", "Christian"]:
        if element in datation:
            for element_cent in [("1st", 1, 100),
                            ("2nd", 101, 200),
                            ("3rd", 201, 300),
                            ("4th", 301, 400),
                           ]:
                if element_cent[0] in datation:
                    dating = [datation, element_cent[1], element_cent[2], None, int(np.average([element_cent[1], element_cent[2]])), "cent", "AD"]
                    break
                else:
                    try:
                        post = re.search('(after\s|post\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                        post = int(post[1])
                        dating = [datation, post, None, None, None, "post", "AD"]
                    except:
                        try:
                            date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                            post = int(date_both[0])
                            ante = int(date_both[2])
                            dating = [datation, post, ante, None, int(np.average([post, ante])), "range", "AD"]
                        except:
                            try:
                                date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
                                date_exact = int(date_exact.group(0))
                                dating = [datation, None, None, date_exact, date_exact, "exact", "AD"]
                            except:
                                dating = [datation, None, None, None, None, "unprecise", "AD"]
                                
            break
        else:
            for element_cent in [(" V ", -500, -401), 
                            (" V/IV ", -500, -301),
                            (" IV ", -400, -301),
                            (" III ", -300, -201),
                            (" II ", -200, -101),
                            (" I ", -100, -1),
                            ("5th", -500,  -401),
                            ("4th", -400, -301),
                            ("3rd", -300, -201),
                            ("2nd", -200, -101),
                            ("1st", -100, -1)]:
                if element_cent[0] in datation:
                    ### if there as an element indicating century
                    dating = [datation, element_cent[1], element_cent[2], None, int(np.average([element_cent[1], element_cent[2]])), "cent", "BC"]
                    break
                else:
                    try:
                        ante = re.search('(ante\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                        ante = int(ante[1]) * -1
                        dating =  [datation, None, ante, None, None, "ante", "BC"]
                    except:
                        try:
                            post = re.search('(post\s|after\s)(\d+)', datation, flags=re.IGNORECASE).groups()
                            post = int(post[1]) * -1
                            dating = [datation, post, None, None, None, "post", "BC"]
                        except:
                            try:
                                date_both = re.search('(\d+)(\-)(\d+)', datation, flags=re.IGNORECASE).groups()
                                post = int(date_both[0]) * -1 
                                ante = int(date_both[2]) * -1
                                dating = [datation, post, ante, None, int(np.average([post, ante])), "range", "BC"]
                            except:
                                try:
                                    date_exact = re.search('\d+', datation, flags=re.IGNORECASE)
                                    date_exact = int(date_exact.group(0)) * -1
                                    dating = [datation, None, None, date_exact, date_exact, "exact", "BC"]
                                except:
                                        dating = [datation, None, None, None, None, "string", "BC"]
    for reign in [("reign of Hadrian", 117, 138),
                  ("reign of Justinian", 527, 565),
                  ("reign of Ant. Pius", 138, 161),
                  ("reign of Augustus", -27, 19),
                  ("reign of Tiberius", 14, 37)]:
        if reign[0] in datation:
            avr = int(np.average([reign[1], reign[2]]))
            if avr < 0:
                period = "BC"
            else:
                period = "AD"
            dating = [datation, reign[1], reign[2], None, avr, "range", period]
            break
    return dating 